In [1]:
from IPython.core.display import display, HTML, Javascript
from string import Template
import json, os
import pandas as pd
import numpy as np

In [2]:
notebook_path = os.path.abspath("Composite_Visualization.ipynb")

In [3]:
arrest_2020 = os.path.join(os.path.dirname(notebook_path), "data/", "NYPD_Arrest_Data__Year_to_Date_CLEANED.csv")
arrest = pd.read_csv(arrest_2020)
del arrest['Unnamed: 0']
def get_month(row):
    arr = row['ARREST_DATE'].split('/')
    return arr[2] + '-' + arr[0]
arrest['MONTH'] = arrest.apply(lambda row: get_month(row), axis=1)
arrest_aggregated = arrest.groupby('MONTH').size().reset_index(name='counts')
data_arrest = arrest_aggregated.to_dict(orient='records')
arrest_aggregated.head()

,MONTH,counts
0,2020-01,16171
1,2020-02,15472
2,2020-03,13179
3,2020-04,8295
4,2020-05,13581


In [4]:
labor_2020 = os.path.join(os.path.dirname(notebook_path), "data/", "nychist_nonfarm.csv")
labor = pd.read_csv(labor_2020)
del labor['AREA']
del labor['SERIESCODE']
del labor['AREANAME']
del labor['INDUSTRY_TITLE']
del labor['ANNUAL']
labor_filtered = labor.loc[labor['YEAR'] == 2020]
data_labor = labor_filtered.to_dict(orient='records')
labor_filtered.head()

,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
1,2020,4638.9,4675.0,4631.4,3748.5,3795.9,3860.0,3909.5,3970.0,4040.2,4102.4,4129.6,4100.2


In [5]:
gun_crime_2020 = os.path.join(os.path.dirname(notebook_path), "data/", "historic_incident_with_zipcode2020_Cleaned.csv")
gun_crime = pd.read_csv(gun_crime_2020)
def get_month_crime(row):
    arr = row['OCCUR_DATE'].split('/')
    return arr[2] + '-' + arr[0]
gun_crime['MONTH'] = gun_crime.apply(lambda row: get_month_crime(row), axis=1)
crime_aggregated = gun_crime.groupby('MONTH').size().reset_index(name='counts')
data_gun_crime = crime_aggregated.to_dict(orient='records')
crime_aggregated.head(12)

,MONTH,counts
0,2020-01,66
1,2020-02,45
2,2020-03,56
3,2020-04,56
4,2020-05,100
5,2020-06,205
6,2020-07,243
7,2020-08,241
8,2020-09,150
9,2020-10,135


In [6]:
arrest_aggregated.to_csv('arrest_aggregated.csv')
labor_filtered.to_csv('labor_filtered.csv')
crime_aggregated.to_csv('crime_aggregated.csv')

In [7]:
%%javascript
require.config({
    paths: {
        d3: "https://d3js.org/d3.v6.min"
     }
});

require(["d3"], function(d3) {
    console.log(d3);
    window.d3 = d3;
});

<IPython.core.display.Javascript object>

In [8]:
html_template = Template('''
<style> $css_text </style>
<div id="graph-div"></div>
<script> $js_text </script>
''')

In [9]:
css_text = ''''''

In [10]:
js_text_template = Template('''
  var margin = {top: 10, right: 50, bottom: 20, left: 40};
  var visWidth = 900 - margin.left - margin.right;
  var visHeight = 500 - margin.top - margin.bottom;
  
  var svg = d3.select("#$graphdiv")
    .append('svg')
    .attr('width', visWidth + margin.left + margin.right)
    .attr('height', visHeight + margin.top + margin.bottom);  
  
  var g = svg.append('g')
    .attr('transform', `translate(${margin.left}, ${margin.top})`); 
    
  var data_labor = $python_data_labor;   
  var data_arrest = $python_data_arrest;
  var data_crime = $python_data_crime;
  
  var map = new Map([["2020-01","JAN"], ["2020-02","FEB"], ["2020-03","MAR"], ["2020-04","APR"],    ["2020-05","MAY"], ["2020-06","JUN"], ["2020-07","JUL"], ["2020-08","AUG"], ["2020-09","SEP"], ["2020-10","OCT"], ["2020-11","NOV"], ["2020-12","DEC"]])
  
  var months = [];
  for (let k of map.values()) {
    months.push(k);
  }
  
  var data_labor1 = [];
  var base = parseFloat(data_labor[0].JAN);
  var list = Object.entries(data_labor[0]);
  for (var i = 1; i < list.length; i++) {
    var o = {};
    o.MONTH = list[i][0];
    o.change = parseFloat(list[i][1]) / base - 1;
    data_labor1.push(o);
  }  
  
  var base_arrest = data_arrest[0].counts;
  var data_arrest1 = data_arrest.map(d=>{
    var o = {};
    o.MONTH = map.get(d.MONTH);
    o.change = d.counts / base_arrest - 1;
    return o;
  })
  
  var base_crime = data_crime[0].counts;
  var data_crime1 = data_crime.map(d=>{
    var o = {};
    o.MONTH = map.get(d.MONTH);
    o.change = d.counts / base_crime - 1;
    return o;
  })
  
  var x = d3.scalePoint()
    .domain(months)
    .range([0, visWidth]);
  
  
  var y = d3.scaleLinear()
    .domain([-0.8, 3]).nice()
    .range([visHeight, 0]);   
  
  var xAxis = d3.axisBottom(x)
  var yAxis = d3.axisLeft(y)
    .tickFormat(d=>d*100+'%');     
  
  g.append('g')
    .attr('transform', `translate(0,${visHeight})`)
    .call(xAxis);  
  
  g.append('g')
      .call(yAxis)
    .append('text')
      .attr('text-anchor', 'start')
      .attr('dominant-baseline', 'middle')
      .attr('fill', 'black')
      .attr('x', 5)
      .style('font-size', 16)
      .text('% Change since January 2020');    
  
  var line = d3.line()
      .x(d => x(d.MONTH))
      .y(d => y(d.change));  
  
  var color = d3.scaleOrdinal()
    .domain(['labor', 'arrest', 'crime'])
    .range(d3.schemeTableau10);

  g.append('path')
      .attr('stroke', color('crime'))
      .attr('stroke-width', 3)
      .attr('fill', 'none')
      .attr('d', line(data_crime1));  
  
  g.append('path')
      .attr('stroke', color('labor'))
      .attr('stroke-width', 3)
      .attr('fill', 'none')
      .attr('d', line(data_labor1));  
  
  g.append('path')
      .attr('stroke', color('arrest'))
      .attr('stroke-width', 3)
      .attr('fill', 'none')
      .attr('d', line(data_arrest1)); 
  
  g.selectAll('circle')
    .data(data_labor1)
    .join('circle')
      .attr('cx', d => x(d.MONTH))
      .attr('cy', d => y(d.change))
      .attr('r', 6)
      .attr('fill', color('labor'))    
  
  g.selectAll('rect')
    .data(data_arrest1)
    .join('rect')
      .attr('x', d => x(d.MONTH)-5)
      .attr('y', d => y(d.change)-5)
      .attr('width', 10)
      .attr('height', 10)
      .attr('fill', color('arrest'))    

  var arc = d3.symbol().type(d3.symbolTriangle)
        .size(80);
  
  var group = svg.append('g')
    .attr('transform', `translate(${margin.left}, ${margin.top})`);
  
  var triangle = group.selectAll('path')
    .data(data_crime1)
    .enter()
    .append('path')
    .attr('d', arc)
    .attr('fill', color('crime'))
    .attr('transform',d => `translate(${x(d.MONTH)}, ${y(d.change)})`); 
  
  var legendGroup = g.append("g")
      .attr('font-family', 'sans-serif')
      .attr('font-size', 12);
  
  var legendRows = legendGroup.selectAll("g")
    .data(['Shooting Incidents', 'Police Arrests', 'Labor Participation'])
    .join("g")
      .attr("transform", (d, i) => `translate(700, ${15 + i * 2.5 * 15})`);

  legendRows.append("text")
    .attr("dominant-baseline", "middle")
    .attr("x", 20)
    .text(d => d);  
  
  legendGroup.append('path')
    .attr('d', arc)
    .attr('fill', color('crime'))
    .attr('transform',d => `translate(710, 15)`); 
  
   legendGroup.append('rect')
    .attr('x', 705)
    .attr('y', 3.5 * 15 - 6)
    .attr('width', 10)
    .attr('height', 10)
    .attr('fill', color('arrest'));
  
   legendGroup.append('circle')
    .attr('cx', 710)
    .attr('cy', 5.9 * 15)
    .attr('r', 6)
    .attr('fill', color('labor')); 
    
   g.append('line')
    .style("stroke", "gray")
    .style("stroke-width", 3)
    .style('opacity', 0.5)
    .attr('x1', 173)
    .attr('y1', 5)
    .attr('x2', 173)
    .attr('y2', visHeight);
  
   g.append('line')
    .style("stroke", "gray")
    .style("stroke-width", 3)
    .style('opacity', 0.5)
    .attr('x1', 340)
    .attr('y1', 5)
    .attr('x2', 340)
    .attr('y2', visHeight);  
  
  g.append('text')
    .attr('x', 175)
    .attr('y', 25)
    .attr('anchor', 'middle')
    .style('font-size', 10)
    .text('March 16th: lockdown began')
  
  g.append('text')
    .attr('x', 342)
    .attr('y', visHeight - 20)
    .attr('anchor', 'middle')
    .style('font-size', 10)
    .text('May 25th: George Floyed died')      
''')

In [11]:
html_template = Template('''
<style> $css_text </style>
<div id="graph-div"></div>
<script> $js_text </script>
''')
js_text = js_text_template.safe_substitute(python_data_labor=json.dumps(data_labor),
                                           python_data_arrest=json.dumps(data_arrest),
                                           python_data_crime=json.dumps(data_gun_crime),
                                       graphdiv='graph-div')
HTML(html_template.safe_substitute(css_text=css_text, js_text=js_text))